In [ ]:
import os
from dotenv import load_dotenv
from utils import get_oracle_connection, fetch_and_insert_posts_comments, safe_execute
import praw

load_dotenv()

if __name__ == "__main__":
    reddit = praw.Reddit(
    client_id=os.getenv("api-client_id"),
    client_secret=os.getenv("api-client_secret"),
    user_agent="stock_market_scraper",
    username=os.getenv("api-username"),
    password=os.getenv("api-password")
    )

    conn = get_oracle_connection()
    if conn:
        cursor = conn.cursor()


        cursor.execute("SELECT subreddit, last_fetched_utc FROM subreddit_tracker")
        subreddit_data = cursor.fetchall()

        for sub, last_utc in subreddit_data:
            print(f"\nFetching new data for subreddit: {sub}")
            conn, cursor, max_utc = fetch_and_insert_posts_comments(
                reddit, sub, last_fetched_utc=last_utc, conn=conn, cursor=cursor
            )

            if max_utc > last_utc:
                conn, cursor = safe_execute(conn, cursor, """
                    UPDATE subreddit_tracker SET last_fetched_utc = :max_utc WHERE subreddit = :sub
                """, {"max_utc": max_utc, "sub": sub})
                print(f"Updated last_fetched_utc for {sub} to {max_utc}")

        cursor.close()
        conn.close()
        print("\n Data inserted and subreddit_tracker updated successfully!")